In [1]:
import pandas as pd
import pandas_profiling as pp

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.util import set_context, raw_path, comp_path

In [4]:
set_context("titanic")

Files in data directory:
______

titanic/
    clean.pkl
    raw/
        gender_submission.csv
        test.csv
        train.csv
______



In [5]:
tr = pd.read_csv(raw_path("train.csv"))
te = pd.read_csv(raw_path("test.csv"))
tr.shape, te.shape

((891, 12), (418, 11))

In [6]:
col_diff = set(tr.columns).difference(te.columns)
assert len(col_diff) == 1
target_col = col_diff.pop()
target_col

'Survived'

In [7]:
tr["_test"] = False
te["_test"] = True

In [8]:
df = pd.concat([tr, te], sort=True)
df.shape

(1309, 13)

In [9]:
df.sample(10).sort_values("_test")

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,_test
888,NaN,NaN,S,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""",2,889,3,female,1,0.0,W./C. 6607,False
220,16.0,NaN,S,8.0500,"Sunderland, Mr. Victor Francis",0,221,3,male,0,1.0,SOTON/OQ 392089,False
20,35.0,NaN,S,26.0000,"Fynney, Mr. Joseph J",0,21,2,male,0,0.0,239865,False
207,26.0,NaN,C,18.7875,"Albimona, Mr. Nassef Cassem",0,208,3,male,0,1.0,2699,False
204,18.0,NaN,S,8.0500,"Cohen, Mr. Gurshon ""Gus""",0,205,3,male,0,1.0,A/5 3540,False
589,NaN,NaN,S,8.0500,"Murdlin, Mr. Joseph",0,590,3,male,0,0.0,A./5. 3235,False
265,NaN,NaN,S,7.8958,"Lyntakoff, Mr. Stanko",0,1157,3,male,0,NaN,349235,True
163,NaN,NaN,S,7.0000,"Pearce, Mr. Ernest",0,1055,3,male,0,NaN,343271,True
235,20.0,NaN,S,7.8542,"Vendel, Mr. Olof Edvin",0,1127,3,male,0,NaN,350416,True
97,29.0,NaN,S,7.9250,"Makinen, Mr. Kalle Edvard",0,989,3,male,0,NaN,STON/O 2. 3101268,True


In [10]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
_test          1309 non-null bool
dtypes: bool(1), float64(3), int64(4), object(5)
memory usage: 479.5 KB


In [11]:
obj_cols = df.select_dtypes("object").columns.tolist()
for col in obj_cols:
    print(f"{col}: {df[col].nunique()} unique values")
    df[col] = df[col].astype("category")

Cabin: 186 unique values
Embarked: 3 unique values
Name: 1307 unique values
Sex: 2 unique values
Ticket: 929 unique values


In [12]:
df.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000


In [13]:
# pp.ProfileReport(df)

In [14]:
df.to_pickle(comp_path("clean.pkl"))